# Model Predictive Control Rocket Soft-Landing Problem

This notebook is a walkthough of the ALTRO solver for the MPC Rocket Soft-Landing Problem (i.e. the general control challenge of landing a rocket at rest at a designated landing site). In the simplest case, we treat the rocket as a sphere landing on a flat surface with no atmosphere. This approximation is reasonable when the rocket is near the ground and entering the final part of the descent.

**This files focuses on the MPC problem and comparing the result to other solvers. For the simple (non-MPC) application, please visit the <code>RocketLandingSimplified.ipynb</code> file. Thanks!**

In [ ]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()

## MPC Problem Outline

For the MPC application, we must fulfill two critical steps:
1. Solve the optimal trajectory and apply the required controls (usually a short horizon)
2. Use the prior trajectory to warm-start the next solve

In this Jupyter notebook, we will:
1. Set up the rocket soft-landing problem 
2. Solve the initial trajectory optimization problem
3. Execute the control (and propagate forward in time)
4. Warm-Start the next iteration
5. Solve the next trajectory optimization problem
6. Repeat steps 3-5 until arrival at the landing location

# Step 1: Set-Up the Rocket Soft-Landing Problem

## Load Required Packages

The <code>Convex</code> package enables us to write the optimization problem. The <code>ECOS</code> package will solve the trajectory optimization problem. <code>StaticArrays</code>, <code>SparseArrays</code>, and <code>LinearAlgebra</code> are required to set-up the appropriate data types.

Next, <code>Plots, MeshCat, TrajOptPlots, FileIO,</code> and <code>MeshIO</code> packages are used to visualize the results. Note that the plotting sections will take significantly longer to run than the optimization. 

Lastly, <code>DifferentialEquations</code> integrates the dynamics forward as a way to show that the dynamics constraints are satisfied.

Loading all the packages, especially for plotting and the differential equations integrator can be the longest part of the whole notebook.

In [ ]:
using Convex
using ECOS
using StaticArrays, SparseArrays, LinearAlgebra
println("Optimization Packages Ready")

using Plots
pyplot()
println("Base Plotting Packages Ready")

using MeshCat, TrajOptPlots
using FileIO, MeshIO
println("3D Plotting Packages Ready")

import DifferentialEquations.solve, DifferentialEquations.ODEProblem, DifferentialEquations.Tsit5
using BenchmarkTools
println("Differential Equations Ready")

## Define and Create the Rocket Model

Unlike the Cartpole and Quadrotor models, the Rocket Model is not in the <code>RobotZoo.jl</code> package. Instead, here we build off of work found in the repository below to develop a simplified rocket soft-landing scenario (i.e. no second-order cone constraints and no attitude dynamics). https://github.com/danineamati/TrajOptSOCPs.jl

In the simplified case, <code>Rocket</code> only holds the mass and the gravity vector. In the full rocket landing problem, the inertia matrix must also be provided. Separately, the thurster's specific impulse can be used to scale the optimal controls.

In [ ]:
struct rocket_simple
    mass
    isp
    grav
    deltaTime
end

## Detail the Rocket Dynamics

For the simplified rocket problem, we choose to ignore the attitude dynamics. Applying newton's second law or by forming a Lagrangian, we arrive at the following sparse system

$$\frac{dx}{dt} = Ax + Bu + g$$

Where $A = \begin{bmatrix}0 & I\\ 0 & 0 \end{bmatrix}$, $B = \begin{bmatrix}0 \\ \frac{1}{m} I\end{bmatrix}$, and $G = \begin{bmatrix} 0 \\ -g \end{bmatrix}$. For the 3D case, these are $(6 \times 6)$, $(6 \times 3)$, and $(6 \times 1)$ matricies, respectively.

Since the system is so sparse, we can reduce unnecessary computations by writing the equations for the first and second derivatives. We use the notation of $s = [x; y; z]$ (the positions) and $\frac{ds}{dt} = [v_x; v_y; v_z]$ (the velocities). Call $y = [x; u; g]$, then,
$$
\frac{dy}{dt} = \begin{bmatrix}A & B & I\\ 0 & 0 & 0\\ 0 & 0 & 0\end{bmatrix} y = W y
$$
which has solution
$$
y(t) = e^{W t} y0
$$
where W is square. In a discrete system,
$$
y_{k+1} = e^{W Δt} y_k = E y_k
$$
Note that due to all the zeros in W, we get:
$$
x_{k+1} = E_{11} x_k + E_{12} u_k + E_{13} g
$$
$$
u_{k+1} = E_{21} x_k + E_{22} u_k + E_{23} g = u_k
$$
$$
g = E_{31} x_k + E_{32} u_k + E_{33} g = g
$$

Thus, once we calculate the matrix $E$, we extract $E_{11}$, $E_{12}$, and $E_{13}$ to a system of the
form:

$$
x_{k+1} = A_{\text{eff}} \ x_k + B_{\text{eff}} \ u_k + G_{\text{eff}}
$$

In [ ]:
function rocketDynamics(r::rocket_simple, nDim::Int64)

    # Start off with a dense representation bc it is needed for the
    # exponentiation.
    z0 = zeros(nDim, nDim)
    aMat = [z0 I; z0 z0]
    bMat = [z0; -(1/r.mass) * I]
    gMat = [z0; I]

    # Top Row
    abg = [aMat bMat gMat]
    abg_size = size(abg)

    # We need to pad the remaining rows to make a square matrix
    remainingRows = abg_size[2] - abg_size[1]
    abgTot = [abg; zeros(remainingRows, abg_size[2])]

    # Take the exponent e^(W Δt)
    eMat = exp(abgTot * r.deltaTime)

    # Extract the effective matrices
    rowsA = size(aMat, 1)
    colsA = size(aMat, 2)
    colsB = size(bMat, 2)

    aEffMat = sparse(eMat[1:rowsA, 1:colsA])
    bEffMat = sparse(eMat[1:rowsA, colsA + 1:colsA + colsB])
    gEffMat = sparse(eMat[1:rowsA, colsA + colsB + 1:end])

    # Return the three effective matrices
    return aEffMat, bEffMat, gEffMat
end

Now we want to stack these per iteration dynamics into a single matrix from which we will build an affine equality constraint. To do this, we create the helper function below.

Specifically, we get the system:
$$\begin{bmatrix}A & B & -I\end{bmatrix} \begin{bmatrix}x_k \\ u_k \\ x_{k+1}\end{bmatrix} = G$$
which we can stack as
$$\begin{bmatrix}
A & B & -I & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & A & B & -I & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & A & B & -I & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & A & B & -I
\end{bmatrix}
$$
We return the above matrix and the stacked `G` matrix.

In [ ]:
function rocketDynamicsStack(r::rocket_simple, nDim::Int64, NSteps::Int64)
    # First calculate the A and B matrices.
    Ak, Bk, Gk = rocketDynamics(r::rocket_simple, nDim::Int64)
    ABI_unit = [Ak Bk -I]
    colLen = size(ABI_unit, 2)

    # At each step we go down 2 * n rows and move right 3 * n columns.
    sizeCols = 3 * NSteps * nDim + 2 * nDim
    sizeRows = 2 * nDim * NSteps

    # Initialize and empty sparse A matrix and G vector
    AStacked = spzeros(sizeRows, sizeCols)
    GStacked = spzeros(sizeRows, 1)

    # Build up the matrix
    rStart = 1
    cStart = 1
    for k in 1:NSteps
        rEnd = rStart + (2 * nDim - 1)
        cEnd = cStart + colLen - 1

        # First, we calculate that stacked A Matrix
        # println("Accessing $rStart:$rEnd and $cStart:$cEnd")
        AStacked[rStart:rEnd, cStart:cEnd] = ABI_unit

        # Second, we calculate the stacked G vector
        GStacked[rStart:rEnd] = -Gk * r.grav

        # Prepare for next iteration.
        rStart += 2 * nDim
        cStart += 3 * nDim
    end

    # Returned the stacked matrix (A) and vector (G)
    return AStacked, GStacked
end

## Instantiate the Model and Define the Discretization

Feel free to choose <code>N</code> and <code>tf</code> based on your needs.

In [ ]:
mass = 1
isp = 1
grav = [0; 0; -9.81]
deltaTime = 0.1
rocket = rocket_simple(mass, isp, grav, deltaTime)

# Trajectory Discretization
N = 250
tf = N * deltaTime
display(tf)

# Dimensionality
num_controls = 3 # ux, uy, uz
num_states = 6 # x, y, z, vx, vy, vz
var_size = N * (num_controls + num_states) + num_states # Last step does not have a control

# Set Up the Trajectory Variable with Convex.jl
tau = Variable(var_size) # Trajectory

rocket # (Simply for information) Display the struct as the cell output 

## Specify the Initial and Final Conditions

Choose the initial and final points. Note that these must be full state vectors ($[x; y; z; v_x; v_y; v_z]$). The initial point is generally the rocket's current state. The final point is generally the rocket's landing stop. Note that the final velocity of the rocket should **always** be $[0.0; 0.0; 0.0]$ to have a **soft** landing.

In [ ]:
# Initial and Final Conditions
# in m
# The Karman Line (100 km)
const rocketStart = [5.0, 5.0, 20.0, 0.0, 0.0, -15.0]
const rocketEnd = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0];

## Initialize the Trajectory

We choose to initialize the trajectory at a hover (i.e. the rocket trajectory will initially only counter gravity). Note that this is **not** a warm start.

In [ ]:
function initializeTraj(x0::Array{Float64, 1}, xN::Array{Float64, 1},
                        NSteps::Int64)
    if size(x0) != size(xN)
        error("x0 and xN must have the same size. " *
                "Currently: x0 = $x0 and xN = $xN")
    end
    if mod(size(x0, 1), 2) != 0
        error("x0 and xN must have an even number of variables. " *
              "Example, x0 = [sx0; sy0; vx0; vy0]. Currently x0 = $x0.")
    end

    # Calculate the linearly interpolated x values
    xSteps = LinRange(x0, xN, NSteps + 1)
    # Now we want to stack this as a vector in the form
    # [x0; u0; x1; ...; uN-1, xN]
    nDim = Int64(size(x0, 1) / 2)
    sizeXU = 3 * NSteps * nDim + 2 * nDim
    XUfull = zeros(sizeXU, 1)

    # println("Dimensions = $nDim")

    for k in 1:(NSteps + 1)
        iStart = 1 + (k - 1) * (3 * nDim)
        iEnd = iStart + (2 * nDim - 1)
        # println("Accessing ($iStart, $iEnd)")
        XUfull[iStart:iEnd] = xSteps[k]
    end

    return XUfull
end

In [ ]:
initTraj = initializeTraj(rocketStart, rocketStart, N)

set_value!(tau, initTraj);

## Set-Up the LQR Objective Function

For this problem, we choose an LQR Objective Function. One can add waypoints or change the weights to craft different optimal trajectories. See http://roboticexplorationlab.org/TrajectoryOptimization.jl/stable/costfunctions.html for more information.

In [ ]:
Q = 0.025 * Diagonal(I, num_states)
Qf = 1 * Diagonal(I, num_states)
R = 0.25 * Diagonal(I, num_controls);

# Initialize and empty sparse array
QRFull = spzeros(var_size, var_size)

rStart = 1

# Put Q and R along the diagonal
for k in 1:N
    rEnd = rStart + num_states - 1
    QRFull[rStart:rEnd, rStart:rEnd] = Q

    rStart += num_states
    rEnd = rStart + num_controls - 1

    QRFull[rStart:rEnd, rStart:rEnd] = R

    rStart += num_controls
end
rEnd = rStart + num_states - 1
QRFull[rStart:rEnd, rStart:rEnd] = Qf

QRFull = Symmetric(QRFull);

In [ ]:
# Set the Objective Function to the problem
#
try 
    problem = minimize(quadform(tau, QRFull / 2))
catch LAPACKException
    problem = minimize(quadform(tau, QRFull / 2))
end

In [ ]:
spy(QRFull)

## Define the Constraints

For the simple rocket soft-landing problem, we will include 5 constraints:
1. Dynamics Constraints ($A \tau = B$)
2. Initial and Terminal Constraints ($x[0] = x_0$ and $x[N] = x_N$)
2. Crash Prevention Constraint ($z > 0$)
3. Thrust Magnitude Bounds ($||u||_2 < u_{max}$)
4. Thurst Angle Bounds ($||[u_x; u_y]|| ≤ \alpha \ u_z$)

In [ ]:
# Add the Dynamics Constraints
const AStack, BStack = rocketDynamicsStack(rocket, num_controls, N)

problem.constraints += AStack * tau == BStack

In [ ]:
# Add Initial and Terminal Constraints
large_num = 10000

problem.constraints += tau[1:6] == rocketStart
problem.constraints += tau[7:(end - 6)] > -large_num
problem.constraints += tau[(end - 5):end] == rocketEnd

In [ ]:
# Add the Ground Constraint
ground_mat = spzeros(var_size, var_size)

currInd = 3
currRow = 1
while currRow <= N
    ground_mat[currRow, currInd] = 1

    currRow += 1
    currInd += 3 * Int(num_states / 2)
end

problem.constraints += ground_mat * tau > 0

In [ ]:
# Add Thrust Magnitude Bounds

perWeightMax = 2

problem.constraints += norm(tau[7:9], 2) < mass * norm(grav[3]) * perWeightMax

## Select the solver options

For more information, please visit https://github.com/RoboticExplorationLab/Altro.jl/blob/master/README.md for more details on the solver options.

Using the Altro package, we can now specify parameters and solve the problem.

# Step 2: Solve the initial trajectory optimization problem

Show summary is set to <code>true</code> to print the solver speed. In general the solve will take about 23 iterations and 20-40 ms to complete. The constraint violation should be around $10^{-4}$ or better.

In [ ]:
problem

Now we can solve it and show the summary

In [ ]:
solve!(problem, ECOS.Optimizer)

In [ ]:
problem

Access the Trajectory

In [ ]:
display(problem.optval)
display(rocketStart)
result_traj = evaluate(tau)

In [ ]:
kStart = 1
maxK = length(tau)
# println("Max = $maxK")

xList = []
vList = []
uList = []

while kStart < (maxK - 2 * num_controls)
    # println("Accessing $kStart to $(kStart + 3 * nDim - 1)")
    push!(xList, evaluate(tau)[kStart:kStart + num_controls - 1])
    push!(vList, evaluate(tau)[kStart + num_controls:kStart + 2 * num_controls - 1])
    push!(uList, evaluate(tau)[kStart + 2 * num_controls:kStart + 3 * num_controls - 1])

    kStart = kStart + 3 * num_controls
end

In [ ]:
xs = [x[1] for x in xList]
ys = [x[2] for x in xList]
zs = [x[3] for x in xList];

In [ ]:
plot(xs, zs)

In [ ]:
plot3d(xs, ys, zs)

In [ ]:
uxs = [u[1] for u in uList]
uys = [u[2] for u in uList]
uzs = [u[3] for u in uList];

In [ ]:
plot(uxs)
plot!(uys)

In [ ]:
plot(-uzs)

## Step 3: Integrate the Dynamics to Verify the Dynamics Constraint

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>.

In [ ]:
num_timesteps = 3

get_controls(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
f(y,p,t) = dynamics(model, y, get_controls(t))

states_y0 = x0
tspan = (0.0, dt * num_timesteps)
controls_p0 = get_controls(tspan[1])

# Use the DifferentialEquations.jl Package
ode_prob = ODEProblem(f, states_y0, tspan, controls_p0)
ode_sol = solve(ode_prob, Tsit5(), reltol=1e-10, abstol=1e-10);

## Step 4: Warm-Start the next iteration

In [ ]:
# warm_start_states = X[num_timesteps:end]
warm_start_controls = U[num_timesteps:end]
x0 = ode_sol.u[end]

N = size(warm_start_controls, 1) + 1
tf = tf - dt * num_timesteps
dt = tf / (N - 1)

## Step 5 & 6: Solve the Problem and Repeat

Now that we've seen each step in detail, let's put it all into a function and repeat it until landing.

In [ ]:
function diffEq_Model(model::Rocket, U, dt)
    get_controls(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
    f(y,p,t) = dynamics(model, y, get_controls(t))

    states_y0 = x0
    tspan = (0.0, dt * num_timesteps)
    controls_p0 = get_controls(tspan[1])

    # Use the DifferentialEquations.jl Package
    ode_prob = ODEProblem(f, states_y0, tspan, controls_p0)
    ode_sol = solve(ode_prob, Tsit5(), reltol=1e-10, abstol=1e-10);

    return ode_sol.u[end]
end

function MPC(model::Rocket, solver, x0, U_init, N, tf)
    
#     println("Setting Up Problem")
    # Reset the objective function
    obj = LQRObjective(Q,R,Qf,xf,N)
    
    # Create Empty ConstraintList
    conSet = ConstraintList(n,m,N)

    # Bounds
    ground_level = 0.0 # Crash Prevention Constraint
    umax = 11 * model.mass # Simplified Max Thrust Constraint
    theta = 20 # deg. Max Angle Constraint
    ulateral_max = umax * sind(theta)

    # To ignore constraints 2 and 3, use the code below
    # bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
    #                                   -Inf, -Inf, -Inf])

    # To include constraints 2 and 3, use the code below
    bnd = BoundConstraint(n,m, x_min=[-Inf, -Inf, ground_level,
                                      -Inf, -Inf, -Inf],
                               u_min=[-ulateral_max, -ulateral_max, 0   ],
                               u_max=[ ulateral_max,  ulateral_max, umax])
    add_constraint!(conSet, bnd, 1:N-1)

    # Goal Constraint that the rocket must reach the landing site.
    goal = GoalConstraint(xf)
    add_constraint!(conSet, goal, N)
    
    # Package the objective and constraints into a "problem" type
    prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet);
    
#     println("Initial Controls")
    # Set the current controls (ideally warm-started)
    initial_controls!(prob, U_init)
    rollout!(prob);
    
#     println("Solving")
    # Solve the new trajectory
    a_s = ALTROSolver(prob, opts)
    solve!(a_s);
    
#     println("Accessing")
    # Access the states and controls
    X = states(a_s)
    U = controls(a_s)
    
#     println("Propogate System")
    # Use the dynamics model (and any disturbances) and propogate forward in time
    x0 = diffEq_Model(model, U, dt)
    
#     println("Warm-Start Next Iteration")
    U_new = U[num_timesteps:end]
    N = size(U_new, 1) + 1
    tf = tf - dt * num_timesteps
    
    # Warm-Start the Next Iteration
    return x0, U_new, N, tf
    
end

In [ ]:
a = MPC(model, ALTROSolver, x0, warm_start_controls, N, tf);

Execute the MPC forward until landed

In [ ]:
completion_threshold = 1e-4

opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

saved_states = []

while norm(x0 - xf) > completion_threshold
    x0, warm_start_controls, N, tf = MPC(model, ALTROSolver, x0, warm_start_controls, N, tf)
    
    println("\nCurrently at time $tf remaining and $(norm(x0 - xf)) away.")
    println("$N Steps at $tf s left. Hence, dt = $(tf / (N-1))")
    
    # If the timespan is too low (e.g. due to disturbances)
    tf = max(0.5, tf)
    
    push!(saved_states, x0)
end

In [ ]:
xs = [x[1] for x in saved_states]
zs = [x[3] for x in saved_states];

## Plot the trajectory!

In [ ]:
titleText = "Rocket Soft-Landing Trajectory \n" *
        "starting at an initial $(x0[6]) m/s plummet"
filename = "RocketGroundThrustBound_BarelyFeasible3"

plt_x = plot(xs, zs, label = "Trajectory")
xlabel!("x")
ylabel!("z (height)")
title!(titleText)

In [ ]:
plt_ux = plot(uxs, label = "Ux Control")
hline!([-ulateral_max, ulateral_max], linecolor = :grey, linestyle = :dash,
                label = "Max Lateral Thrust")
# xlabel!("time (s)")
ylabel!("control (N)")
title!("Controls over Time")

plt_uz = plot(uzs, label = "Uz Control")
hline!([umax], linecolor = :grey, linestyle = :dash, label = "Max Thrust")
xlabel!("time (s)")
ylabel!("control (N)")

plt_u = plot(plt_ux, plt_uz, layout = (2, 1))

Below is a GIF of the trajectory. This will take several seconds to generate.

In [ ]:
xmin = minimum(xs)
xmax = maximum(xs)

zmin = minimum(zs)
zmax = maximum(zs)

@gif for i in 1:N
    plt = plot(xs[1:i], zs[1:i], label = "Trajectory", legend = :topleft)
    xlabel!("x")
    ylabel!("z (height)")
    xlims!(xmin,xmax)
    ylims!(zmin,zmax)
    title!(titleText)
end

## Lastly, let's visualize the results in 3D with Meshcat

To do this, we need to provide a Mesh to <code>MeshCat</code>. This is due with <code>FileIO</code> and <code>MeshIO</code>.

In [ ]:
vis = Visualizer() # Creates a MeshCat Window
render(vis) # Renders the window below

In [ ]:
function TrajOptPlots._set_mesh!(vis, model::Rocket)
    obj = joinpath(@__DIR__, "SimpleRocket.obj")
    quad_scaling = 0.085
    robot_obj = FileIO.load(obj)
    robot_obj.vertices .*= quad_scaling
    mat = MeshPhongMaterial(color=colorant"lightgrey")
    setobject!(vis["geom"], robot_obj, mat)
end

TrajOptPlots.set_mesh!(vis, model)

In [ ]:
Base.position(::Rocket, x::StaticVector) = SA[x[1], x[2], x[3]]
RobotDynamics.orientation(::Rocket, x::StaticVector) = UnitQuaternion(0,0,0,1)

visualize!(vis, altro);

## Integrate the Dynamics to Verify the Dynamics Constraint

In Matlab, we would use ODE 45, but in the <code>DifferentialEquations.jl</code> package we can either (1) just let the <code>solve</code> method figure it out or (2) specify a solver such as <code>Tsit5</code>.

The controls change over time and are discretized. Below, a zero-order hold and a first-order hold are implemented for comparison. **We used a zero-order hold in the optimization so a zero-order hold should be used here.**

In [ ]:
get_controls_zero_order_hold(t) = U[min(Int(floor(t/dt)) + 1, size(U,1))]
f(y,p,t) = dynamics(model, y, get_controls_zero_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_zero_order_hold(tspan[1])

ode_prob_zoh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_zoh = solve(ode_prob_zoh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
function get_controls_first_order_hold(t)
    # dt and U are both global variables
    tn = t/dt
    
    t_below = Int(max(1, floor(tn)))
    t_above = Int(min(size(U, 1), ceil(tn)))
    
    if t_below >= t_above
        return U[t_below]
    end
    
    slope = (U[t_above] - U[t_below])
    return slope * (tn - t_below) + U[t_below]

end


f(y,p,t) = dynamics(model, y, get_controls_first_order_hold(t))

states_y0 = x0
tspan = (0.0, tf)
controls_p0 = get_controls_first_order_hold(tspan[1])

ode_prob_foh = ODEProblem(f, states_y0, tspan, controls_p0)
@btime solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt)
ode_sol_foh = solve(ode_prob_foh, Tsit5(), reltol=1e-10, abstol=1e-10,saveat=dt);

In [ ]:
err_zoh = norm.(ode_sol_zoh.u - X[1:(Int(tspan[2]/dt)+1)])
err_foh = norm.(ode_sol_foh.u - X[1:(Int(tspan[2]/dt)+1)])

plot(ode_sol_zoh.t, err_zoh, yaxis = :log, label = "Zero-Order Hold")
plot!(ode_sol_foh.t, err_foh, yaxis = :log, label = "First-Order Hold")
xlabel!("Time (s)")
ylabel!("Dynamics Error")
title!("Comparison in State Vector expected from ALTRO Solver \n to Integrated Dynamics Model")